In [ ]:

!pip install bert-for-tf2
!pip install sentencepiece

In [ ]:

%tensorflow_version 2.x
 
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.3.0
Hub version:  0.9.0


In [ ]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)


In [ ]:

MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                    name="segment_ids")

In [ ]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))
 
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [ ]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])


In [ ]:
FullTokenizer=bert.bert_tokenization.FullTokenizer
 
vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
 
do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
 
tokenizer=FullTokenizer(vocab_file,do_lower_case)
 
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
url="https://raw.githubusercontent.com/Sooryak12/FakeReviewDetectionLSTM/master/selected.txt"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install nlppreprocess

In [ ]:
df=pd.read_csv(url,delimiter="\t")
df.head(10)

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...
5,6,__label1__,3,N,Health & Personal Care,B00686HNUK,Tobacco Pipe Stand - Fold-away Portable - Ligh...,not sure,I'm not sure what this is supposed to be but I...
6,7,__label1__,4,N,Toys,B00NUG865W,ESPN 2-Piece Table Tennis,PING PONG TABLE GREAT FOR YOUTHS AND FAMILY,Pleased with ping pong table. 11 year old and ...
7,8,__label1__,4,Y,Beauty,B00QUL8VX6,Abundant Health 25% Vitamin C Serum with Vitam...,Great vitamin C serum,Great vitamin C serum... I really like the oil...
8,9,__label1__,4,N,Health & Personal Care,B004YHKVCM,PODS Spring Meadow HE Turbo Laundry Detergent ...,wonderful detergent.,I've used tide pods laundry detergent for many...
9,10,__label1__,1,N,Health & Personal Care,B00H4IBD0M,"Sheer TEST, Best Testosterone Booster Suppleme...",WARNING: do not waste your money on this,Everybody wants to fall for their promises. Bu...


In [ ]:
df["LABEL"]=df["LABEL"].replace(['__label2__','__label1__'],['0','1'])
dfs=df[["REVIEW_TEXT","LABEL","PRODUCT_CATEGORY"]]
dfs = dfs.rename(columns={'REVIEW_TEXT': 'reviews',"LABEL":"label","PRODUCT_CATEGORY":"cat"})
dfs["label"]=dfs["label"].astype(np.float32)

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)
dfs["reviews"]=dfs["reviews"].apply(preprocess_text)
dfs["reviews"]=dfs["reviews"].apply(remove_tags)
dfs["reviews"]=dfs["reviews"].str.lower()

In [ ]:
from nlppreprocess import NLP
obj=NLP(remove_stopwords=True,remove_numbers=True)
dfs["reviews"]=dfs["reviews"].apply(obj.process)
dfs["len_review"]=dfs["reviews"].apply(lambda x :len(x.split()))
dfse=dfs[dfs["len_review"]<150]
dfse.reset_index(drop=True,inplace=True)

In [ ]:
dfse[["label0","label1"]]=pd.get_dummies(dfse["label"])

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)
 
  return ids,masks,segments
 
def create_input_array(sentences):
 
  input_ids, input_masks, input_segments = [], [], []
 
  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)
 
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)
 
  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [ ]:
""""x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
x = tf.keras.layers.Dropout(0.2)(x)
out= tf.keras.layers.Dense(1, activation="sigmoid", name="dense_output")(x)
 
model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
 
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])"""

'"x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)\nx = tf.keras.layers.Dropout(0.2)(x)\nout= tf.keras.layers.Dense(1, activation="sigmoid", name="dense_output")(x)\n \nmodel = tf.keras.models.Model(\n      inputs=[input_word_ids, input_mask, segment_ids], outputs=out)\n \nmodel.compile(loss=\'binary_crossentropy\',\n                  optimizer=\'adam\',\n                  metrics=[\'accuracy\'])'

In [ ]:
inputs=create_input_array(dfse["reviews"])
train_y=dfse["label"]
 


100%|██████████| 20160/20160 [00:11<00:00, 1719.49it/s]


In [ ]:
!pip install transformers

In [ ]:
#r=model.fit(inputs,train_y,epochs=5,batch_size=32,validation_split=0.2,shuffle=True,verbose=1 )

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertModel #TFBertForSequenceClassification
import tensorflow as tf

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 3

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                        num_labels=1,hidden_dropout_prob=0.3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy,But i used Binary Cross Entropy
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
r=model.fit(inputs,train_y,epochs=3,batch_size=32,validation_split=0.2,shuffle=True,verbose=1 )

Epoch 1/3
504/504 [==============================] - ETA: 0s - loss: 0.6027 - binary_accuracy: 0.6344WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0081s vs `on_test_batch_end` time: 0.2757s). Check your callbacks.


504/504 [==============================] - 461s 914ms/step - loss: 0.6027 - binary_accuracy: 0.6344 - val_loss: 1.0592 - val_binary_accuracy: 0.4700
Epoch 2/3
504/504 [==============================] - 461s 914ms/step - loss: 0.5078 - binary_accuracy: 0.7348 - val_loss: 1.3181 - val_binary_accuracy: 0.4913
Epoch 3/3
504/504 [==============================] - 461s 914ms/step - loss: 0.3403 - binary_accuracy: 0.8486 - val_loss: 2.0020 - val_binary_accuracy: 0.3743


**BertModel 1 with one output Neuron -
Epoch 3/3 loss: 0.3403 - binary_accuracy: 0.8486 - val_loss: 2.0020 - val_binary_accuracy: 0.3743**

In [ ]:
train_y=dfse["label"]

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                        num_labels=2,hidden_dropout_prob=0.3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy,But i used Binary Cross Entropy
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_306']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
r=model.fit(inputs,train_y,epochs=5,batch_size=32,validation_split=0.2,shuffle=True,verbose=1 )

Epoch 1/5
504/504 [==============================] - ETA: 0s - loss: 0.6124 - binary_accuracy: 0.6419WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0086s vs `on_test_batch_end` time: 0.2727s). Check your callbacks.


504/504 [==============================] - 462s 918ms/step - loss: 0.6124 - binary_accuracy: 0.6419 - val_loss: 1.3862 - val_binary_accuracy: 0.1741
Epoch 2/5
341/504 [===================>..........] - ETA: 2:16 - loss: 0.5657 - binary_accuracy: 0.6940

KeyboardInterrupt: ignored

**Bert Model 2 with 2 output neurons
Labels are Segregated .
Model Training stopped due lease testing accuracy in 1st epoch.**

In [ ]:
train_y=dfse[["label0","label1"]]
 

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                        num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_151']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
r=model.fit(inputs,train_y,epochs=3,batch_size=32,validation_split=0.2,shuffle=True,verbose=1 )

Epoch 1/3
504/504 [==============================] - ETA: 0s - loss: 0.6033 - binary_accuracy: 0.6506WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0090s vs `on_test_batch_end` time: 0.2745s). Check your callbacks.


504/504 [==============================] - 464s 921ms/step - loss: 0.6033 - binary_accuracy: 0.6506 - val_loss: 0.7742 - val_binary_accuracy: 0.4849
Epoch 2/3
504/504 [==============================] - 461s 915ms/step - loss: 0.5059 - binary_accuracy: 0.7396 - val_loss: 0.7856 - val_binary_accuracy: 0.5166
Epoch 3/3
504/504 [==============================] - 462s 917ms/step - loss: 0.3521 - binary_accuracy: 0.8407 - val_loss: 1.4328 - val_binary_accuracy: 0.4435


**BERT Model 3 with 2 output neurons **
**Epoch 3/3  loss: 0.3521 - binary_accuracy: 0.8407 - val_loss: 1.4328 val_binary_accuracy: 0.4435**